In [1]:
#Importing Library and setting environment path
import os
import sys
#set the path 
os.environ['SPARK_HOME'] = "C:/spark-2.0.2-bin-hadoop2.7"
sys.path.append("C:/park-2.0.2-bin-hadoop2.7/bin")
sys.path.append("C:/spark-2.0.2-bin-hadoop2.7/python")
sys.path.append("C:/spark-2.0.2-bin-hadoop2.7/python/pyspark")
sys.path.append("C:/spark-2.0.2-bin-hadoop2.7/python/pyspark/lib")
sys.path.append("C:/spark-2.0.2-bin-hadoop2.7/python/pyspark/lib/pyspark.zip")
sys.path.append("C:/spark-2.0.2-bin-hadoop2.7/python/pyspark/lib/py4j-0.10.3-src.zip")
sys.path.append("C:/Program Files/Java/jre1.8.0_77/bin")



In [2]:
# Streaming Program
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

ImportError: No module named py4j.java_gateway

In [ ]:
sc

In [ ]:
ssc = StreamingContext(sc,10)

In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
#ssc.checkpoint( "file:///Users/xinyingdu/Google Drive/UMN/Fall/MSBA 6330 Harvesting Big Data/MyProject/checkpoint")

In [ ]:
socket_stream = ssc.socketTextStream("localhost", 5555)

In [ ]:
lines = socket_stream.window( 20 )
# lines = socket_stream

In [ ]:
from collections import namedtuple
fields = ("tag", "count" )
fields2 = ("time", "sentiment" )
Tweet = namedtuple( 'Tweet', fields )
txt = namedtuple( 'txt', fields2 )

In [ ]:
# (lines.map(lambda text: text.split( "|" ) )\
#   .map(lambda word: Tweet( word[0], word[1]))
#   .foreachRDD(lambda rdd: rdd.toDF()
#   .registerTempTable("sentiments")))

In [ ]:
(lines.flatMap( lambda text : text.split( " " ) )
  .filter( lambda word: word.lower().startswith("@") )
  .map( lambda word: ( word.lower(), 1) )
  .reduceByKey( lambda a, b: a + b )
  .map( lambda rec: Tweet( rec[0], rec[1] ) )
  .foreachRDD(lambda rdd: rdd.toDF()
  .sort(desc("count")).limit(10).registerTempTable("tweets")))

In [ ]:
sqlContext

In [ ]:
ssc.start()
#ssc.awaitTermination()

Tweet(tag=u'#GrabYourWalletMon Dec 12 02:52:21 +0000 2016', count=0.4404)

In [ ]:
import numpy as np 
import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go
import plotly 
plotly.tools.set_credentials_file(username='xinyingdu', api_key='Ssz6FAoJ34JBnxEy7ltn')


In [ ]:
token_1 = 'taj2aa9og4'
# token_2 = 'e7sfqqk6vf'
stream_id1 = dict(token=token_1, maxpoints=1000)
# stream_id2 = dict(token=token_2, maxpoints=60)

In [ ]:
trace1 = go.Scatter(x=[], y=[], stream=stream_id1, name='Popularity')
#trace2 = go.Scatter(x=[], y=[], stream=stream_id2, yaxis='y2', name='Sentiment', marker=dict(color='rgb(148, 103, 189)'))

data = go.Data([trace1])
layout = go.Layout(
    title='Ebay Twitter Stream',
    yaxis=dict(
        title= 'Popularity'))
#     ),
#     yaxis2=dict(
#         title='Sentiment',
#         titlefont=dict(
#             color='rgb(148, 103, 189)'
#         ),
#         tickfont=dict(
#             color='rgb(148, 103, 189)'
#         ),
#         overlaying='y',
#         side='right'
#     )
#)

fig = go.Figure(data=data, layout=layout)

# Send fig to Plotly, initialize streaming plot, open new tab
py.iplot(fig, filename='ebay-twitter-streaming')

In [ ]:
s_1 = py.Stream(stream_id=token_1)
#s_2 = py.Stream(stream_id=token_2)

In [ ]:
s_1.open()
#s_2.open()

import time
import datetime
import numpy as np
#count = 0
while True:
    time.sleep( 6 )
    # getting twitter count 
    pop_count = sqlContext.sql( "select sum(count) as pop from tweets WHERE tag LIKE '%ebay%'")
    pop_count_df = pop_count.toPandas()
    x = datetime.datetime.now()
    y = pop_count_df['pop'][0]
    
    # getting sentiments
#     senti = sqlContext.sql( "select mean(count) as senti from sentiments")
#     senti_df = senti.toPandas()
#     x1 = datetime.datetime.now()
#     y1 = senti_df.senti[0]
        # Send data to your plot
    s_1.write(dict(x=x, y=y))
#     s_2.write(dict(x=x1, y=y1)) 
s_1.close()
#s_2.close()
#count = count + 1

In [ ]:
ssc.stop(False)